In [1]:
from openai import OpenAI
from apikey import api_key
from myTools import *
client = OpenAI(api_key=api_key)

In [22]:
docu =read_file("docs/envision-brief.md")    
challenge = read_file("mychallenges/"+"c000"+".md")

# decompose challenge into question and prof_answer

question,prof_answer,references=decompose_challenge(challenge)
ref_str=create_ref(references)
print(question,prof_answer,ref_str)

define a text literal called greetings with value "Hello" and display greetings on the dashboard as a label.   ```envision
greeting = "Hello" // define the text literal
show label greeting // show the text literal as a label. There should be no 'with' !
``` +++
title = "Envision Language"
url = "language"
description = "Envision is the Domain-Specific Language (DSL) engineered by Lokad for the specific purpose of the predictive optimization of supply chains. This document is not intended for complete programming beginners, but rather for an audience already familiar with basic programming patterns like Microsoft Excel formulas."
weight = 1
alwaysopen = false
+++

Envision is the Domain-Specific Language (DSL) engineered by Lokad for the specific purpose of the predictive optimization of supply chains. This document is not intended for complete programming beginners, but rather for an audience already familiar with basic programming patterns like Microsoft Excel formulas.

Envision has 

In [9]:
coder_personality="You are a proficient coder in the Domain Specific Language called Envision. \
    Your task is to generate response to the given challenge. \
    Some challenges will ask you to generate Envision code,\
    others will ask you to explain given code or answer questions related to the Envision language. \
    Do not output any intermediate thinking or explanation, only give the final answer.\
    Here is the documentation of Envision:\
    ### Documentation\n" + docu
coder_prompt=question

coder_response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": coder_personality},
        {"role": "user", "content": coder_prompt}
    ],
    max_tokens=1000,  # Adjust the number of tokens based on your needs
    temperature=0.4,
)
stud_sentence=coder_response.choices[0].message.content
print(stud_sentence)


```envision
greetings = "Hello"
show label greetings
```


In [48]:

# this personality sticks more to professor's answer.

judge_personality_teacherAuthority="Your goal is to judge the correctness of STUDENT ANSWER, as an answer to the QUESTION.\
In order to judge the STUDENT ANSWER, you are given the PROFESSOR ANSWER with a piece of related documentation.\
Your main job is not to check the syntax correctness, but the logical correctness.\
If the STUDENT ANSWER does not treat the QUESTION logically, it is UNACCEPTABLE.\
Pay special attention to the comments in the PROFESSOR ANSWER. If these comments include\
a rule and if the STUDENT ANSWER violates it, this is UNACCEPTABLE.\
If in the show command, the STUDENT ANSWER add or omit a print position (like a1b2) compared to the PROFESSOR ANSWER, ignore this: this is always ACCEPTABLE.\
The use of extra variable or table to temporarily contain a intermediate quantity is ACCEPTABLE.\
Differences in variable names, column names, table names and label names etc. shall systematically be ACCEPTABLE! \
There are sometimes various ways or logics to treat the same QUESTION, and this is ACCEPTABLE, as long as the goal of the QUESTION is achieved.\
Let's think aloud step by step before making your judgement. Tell each ACCEPTABLE or UNACCEPTABLE point. \
At the end of your output, you should judge 0 if there is anything UNACCEPTABLE (even only 1 mark of UNACCEPTABLE) in the STUDENT ANSWER;\
and judge 1 if everything is ACCEPTABLE. End your judgment by the digit either 0 or 1. \
Here is the piece of related documentation : \n ## DOCUMENTATION\n" + ref_str

judge_prompt = "### QUESTION: "+question+"\n### PROFESSOR ANSWER: "+prof_answer+"\n### STUDENT ANSWER: "+stud_sentence

# Generate a response from the chatbot
judge_response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": judge_personality_teacherAuthority},
        {"role": "user", "content": judge_prompt}
    ],
    max_tokens=800,  # Adjust the number of tokens based on your needs
    temperature=0.1,
)
print(judge_prompt)
# Print the generated response
print(judge_response.choices[0].message.content)


### QUESTION: define a text literal called greetings with value "Hello" and display greetings on the dashboard as a label.  
### PROFESSOR ANSWER: ```envision
greeting = "Hello" // define the text literal
show label greeting // show the text literal as a label. There should be no 'with' !
```
### STUDENT ANSWER: ```envision
greetings = "Hello"
show label greetings
```
The student's answer is ACCEPTABLE.

- The student correctly defines a text literal called `greetings` with the value "Hello".
- The student correctly displays the `greetings` text literal on the dashboard as a label.

Therefore, the student's answer fulfills the requirements of the question and is logically correct.

The student's answer is ACCEPTABLE.

### Judgement: 1


In [27]:
verifier_personality="Your task is to summarize the input given by the judge:\
    - If the judge has found nothing unacceptable, you should output 1.\
    - If the judge has found anything unacceptable, you should output 0.\
    - Focus on the last line of the judge's sentence: if it has already announced the final judgement, you should output the same (0 or 1).\
    Do not output anything other than pur digit 0 or 1, without font, punctuation or any special character."
verifier_response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": verifier_personality},
        {"role": "user", "content": judge_response.choices[0].message.content}
    ],
    max_tokens=800,  # Adjust the number of tokens based on your needs
    temperature=0.05,
)
print(verifier_response.choices[0].message.content)


1


In [11]:
# extract the 'real' code from the student answer (cut away the '''envision bit at the start and end)
def extract_code(stud_sentence):
    lines = stud_sentence.strip().split('\n')
    return '\n'.join(lines[1:-1])
print(extract_code(stud_sentence))

greetings = "Hello"
show label greetings


In [12]:
import requests
# send code to online compiler and check if it compiles

def check_compilation(script):
    url = "https://try.lokad.com/w/script/trycompile"
    payload = {
        "Script": script
    }

    try:
        # Send POST request
        response = requests.post(url, json=payload)

        # Check for successful response
        if response.status_code == 200:
            result = response.json()
            if result["IsCompOk"]:
                return True
            else:
                print("Compilation Failed!")
                for message in result["CompMessages"]:
                    print(f"Error: {message['Text']} (Line: {message['Line']}, Start: {message['Start']}, Length: {message['Length']}, Severity: {message['Severity']})")
                    return False
        else:
            print("Error: Unable to reach the compilation service.")
            return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

        

# Example usage
check_compilation(extract_code(stud_sentence))


True

In [36]:
# for each question, try 3 generation-compilations.
# if compiles, further check with judge.
def pipeline_verify(challenge,coder_personality,judge_personality=judge_personality_teacherAuthority):

    question,prof_answer,references=decompose_challenge(challenge)
    ref_str=create_ref(references)    
    n_tries=3

    for compile_try in range(n_tries):
        coder_prompt=question
        coder_response = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=[
                {"role": "system", "content": coder_personality},
                {"role": "user", "content": coder_prompt}
            ],
            max_tokens=1000,  # Adjust the number of tokens based on your needs
            temperature=0.2,
        )
        stud_sentence=coder_response.choices[0].message.content
        if (question.split("\n")[0] ==\
        '# this question expects a textual answer and not generation of code. #'):
            print('# theoretical question, no compile.')
            break
        if(check_compilation(extract_code(stud_sentence))):
            print('# compile ok')
            break
        elif (compile_try==n_tries-1):
            print( "# too many failures !")
            print('# badcode:\n'+extract_code(stud_sentence))
            return stud_sentence,"too many failures !",False

    judge_prompt = "### QUESTION: "+question+"\n### PROFESSOR ANSWER: "+prof_answer+"\n### STUDENT ANSWER: "+stud_sentence
    judge_response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": judge_personality+ref_str},
            {"role": "user", "content": judge_prompt}
        ],
        max_tokens=800,  # Adjust the number of tokens based on your needs
        temperature=0.2,
    )
    judge_sentence=judge_response.choices[0].message.content
    verifier_response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": verifier_personality},
        {"role": "user", "content": judge_sentence}
    ],
    max_tokens=800,  # Adjust the number of tokens based on your needs
    temperature=0.05,
)
    judge_decision=verifier_response.choices[0].message.content=='1'
    print ('# judge_decision:',judge_decision)
    if not judge_decision:
        print('# badcode:\n',extract_code(stud_sentence))
        print('# judge explanation:\n',judge_sentence)
    return stud_sentence,judge_sentence,judge_decision
# a all-in-one function to score a model on a list of challenges
def pipeline_score_allchallenge(indexes,coder_personality):
    challenges=[read_file("mychallenges/"+index+".md") for index in indexes]
    score=0
    for i in range(len(challenges)):
        challenge=challenges[i]
        print('\n### verifying challenge No. '+indexes[i])
        _,_,judge_decision=pipeline_verify(challenge,coder_personality)
        if (judge_decision): score+=1
    print('correct:'+str(score)+' out of '+str(len(challenges))+', '+str(score/len(challenges)*100)+'%')  

In [49]:
import os
indexes = [f.split('.')[0] for f in os.listdir('mychallenges') if f.endswith('.md') and f!='description.md']
pipeline_score_allchallenge(indexes,coder_personality)


### verifying challenge No. c000
# compile ok
# judge_decision: True

### verifying challenge No. c001
# compile ok
# judge_decision: True

### verifying challenge No. c002
# compile ok
# judge_decision: True

### verifying challenge No. c003
# compile ok
# judge_decision: True

### verifying challenge No. c004
# compile ok
# judge_decision: True

### verifying challenge No. c005
# theoretical question, no compile.
# judge_decision: True

### verifying challenge No. c006
# compile ok
# judge_decision: False
# badcode:
 table T = with
  [| as Name, as Score |]
  [| "Alice", 85 |]
  [| "Bob", 92 |]
  [| "Charlie", 88 |]
  [| "David", 90 |]
  [| "Eve", 87 |]

maxScore = max(T.Score)
bestName = argmax(T.Name, T.Score)

show scalar "Max Score" a1b2 with maxScore
show scalar "Best Name" c1d2 with bestName
# judge explanation:
 The student answer is **UNACCEPTABLE**.

1. The student defined the table columns as "Name" and "Score", while the professor defined them as "name" and "score". This 